# 1. Основы PySpark

**Всегда закрывайте сессию после работы**
```
spark.stop()
```

Установите Java (обязательно!). Скачайте Amazon Corretto 11 (или OpenJDK 11).  
```
# Проверка (в PowerShell):
java -version
```
Установите Python 3.11. Скачайте с официального сайта. При установке отметьте "Add Python to PATH".  
```
python --version
pip --version
```
Установите Hadoop и Spark. Скачайте Spark 3.5.x (предпочтительно) и Hadoop 3.3. Распакуйте архив в C:\spark (или другой путь без пробелов). Скачайте winutils.exe для Hadoop в папку C:\hadoop\bin. Добавьте переменные среды:
```
[System.Environment]::SetEnvironmentVariable("HADOOP_HOME", "C:\hadoop", "Machine")
[System.Environment]::SetEnvironmentVariable("SPARK_HOME", "C:\spark", "Machine")
[System.Environment]::SetEnvironmentVariable("PATH", "$env:PATH;C:\spark\bin;C:\hadoop\bin", "Machine")
```

Установите PySpark и зависимости
```
pip install pyspark pandas jupyter pyarrow
```

Проверка установки. Тест PySpark в Jupyter
```
jupyter notebook
```
В ноутбуке выполните
```
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .appName("Test") \
    .getOrCreate()
df = spark.createDataFrame([(1, "Alice"), (2, "Bob")], ["id", "name"])
df.show()
```
Вывод

| id| name|
| -- | -- |
|  1|Alice|
|  2|  Bob|

**Используемые датасеты**  
[Titanic](https://www.kaggle.com/competitions/titanic/data?select=train.csv "CSV")  
[Amazon Sales Data](https://www.kaggle.com/datasets/karkavelrajaj/amazon-sales-dataset "JSON")  
[COVID-19 Dataset](https://www.kaggle.com/datasets/imdevskp/corona-virus-report "Parquet")  

In [1]:
import os
os.environ['PYSPARK_PYTHON'] = 'python'
os.environ['HADOOP_USER_NAME'] = 'root'  # Обход проверки пользователя

from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Test") \
    .master("local[*]") \
    .config("spark.driver.host", "localhost") \
    .config("spark.executor.memory", "2g") \
    .getOrCreate()

In [2]:
spark = SparkSession.builder \
    .appName("Test") \
    .getOrCreate()
df = spark.createDataFrame([(1, "Alice"), (2, "Bob")], ["id", "name"])
df.show()

+---+-----+
| id| name|
+---+-----+
|  1|Alice|
|  2|  Bob|
+---+-----+



## 1.1. Введение в Spark и RDD

PySpark — это Python API для Apache Spark — высокопроизводительной распределённой платформы для обработки больших данных. Позволяет писать параллельные программы, которые работают на кластерах, но при этом использовать знакомый Python. Spark работает с большими объёмами данных, поддерживает SQL, машинное обучение, потоковую обработку. Jupyter Notebook — удобный инструмент для интерактивного написания и запуска кода, идеально подходит для обучения и прототипирования.

RDD(Resilient Distributed Dataset) - базовая структура данных в Spark. **Неизменяемая:** После создания нельзя изменить, только преобразовать. **Распределённая:** Данные разделены на части (партиции) и обрабатываются параллельно.  
SparkSession – точка входа для работы с Spark (аналог SparkContext в старых версиях).

Ключевые характеристики RDD:
- Resilient (Устойчивость). RDD автоматически восстанавливается при сбоях благодаря lineage (цепочке преобразований). Если часть данных теряется, Spark пересчитывает её заново на основе исходных данных и применённых операций.
- Distributed (Распределённость). Данные разбиваются на партиции (partitions) и распределяются по узлам кластера. Обработка происходит параллельно.
- Dataset (Набор данных). Может содержать данные любого типа (числа, строки, объекты и т. д.). Поддерживает операции: map, filter, reduce, join и др.

Как создаются RDD?  
```
# Из внешних данных (HDFS, S3, локальная файловая система):
rdd = sc.textFile("hdfs://path/to/file.txt")

# Из коллекций в памяти (например, список Python):
rdd = sc.parallelize([1, 2, 3, 4, 5])

# Преобразованием другого RDD (например, map, filter):
rdd2 = rdd.map(lambda x: x * 2)
```

### Пример:

Создание RDD из списка

In [3]:
from pyspark.sql import SparkSession

# Инициализация SparkSession
spark = SparkSession.builder \
    .appName("RDD Basics") \
    .master("local[*]") \
    .getOrCreate()

# Создание RDD из списка чисел
data = [1, 2, 3, 4, 5]
rdd = spark.sparkContext.parallelize(data)

# Трансформация: умножение каждого элемента на 2
rdd_doubled = rdd.map(lambda x: x * 2)

# Действие: сбор результатов
print(rdd_doubled.collect())  # Вывод: [2, 4, 6, 8, 10]

[2, 4, 6, 8, 10]


Фильтрация данных

In [4]:
# Фильтрация чётных чисел
rdd_even = rdd.filter(lambda x: x % 2 == 0)
print(rdd_even.collect())  # Вывод: [2, 4]

[2, 4]


Работа с текстом

In [5]:
text_data = ["Hello Spark", "Hello Python", "Hello World"]
text_rdd = spark.sparkContext.parallelize(text_data)

# Разбиваем строки на слова
words_rdd = text_rdd.flatMap(lambda line: line.split(" "))

# Подсчёт слов
word_counts = words_rdd.countByValue()
print(word_counts)  # Вывод: {'Hello': 3, 'Spark': 1, 'Python': 1, 'World': 1}

defaultdict(<class 'int'>, {'Hello': 3, 'Spark': 1, 'Python': 1, 'World': 1})


## 1.2. DataFrames (основы)

PySpark DataFrame - это распределенная коллекция данных, организованная в именованные столбцы, предоставляемая Apache Spark для работы с большими данными в Python. Это аналог pandas DataFrame, но предназначенный для работы в распределенной среде.

**Основные характеристики**. *Распределенная структура*: Данные разделены между узлами кластера. *Неизменяемость*: DataFrame иммутабельны, все операции создают новые DataFrame. *Ленивые вычисления*: Операции выполняются только при вызове действия (action). *Оптимизация*: Использует Catalyst Optimizer для оптимизации запросов.

Отличие от RDD:
- Оптимизирован для SQL-подобных операций.
- Имеет схему (типы данных колонок).

Основные методы:
- show() – вывод данных.
- printSchema() – вывод структуры.
- select(), filter(), groupBy(), agg().

### Практика

Воспользуемся датасет Titanic (файл train.csv)

In [6]:
spark = SparkSession.builder \
    .appName("Titanic Analysis") \
    .master("local[*]") \
    .getOrCreate()

# Загрузка CSV с заголовком и выводом схемы
df_Titanic = spark.read.csv("data/train.csv", header=True, inferSchema=True)
df_Titanic.printSchema()

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



Исследование данных

In [7]:
# Показать первые 5 строк
df_Titanic.show(5)

# Статистика по числовым колонкам
df_Titanic.describe(["Age", "Fare"]).show()

# Количество выживших и погибших
df_Titanic.groupBy("Survived").count().show()

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| NULL|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| NULL|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05| NULL|       S|
+-----------+--------+------+--------------------+------+----+-----+-----+------

Фильтрация и агрегация. 

In [8]:
from pyspark.sql.functions import avg, max  # Импортируем нужные функции

# Стоимость билета по классам
df_Titanic.groupBy("Pclass").agg(avg("Fare").alias("AvgFare"), max("Fare").alias("MaxFare")).show()

# Средний возраст выживших женщин.
df_Titanic.filter((df_Titanic.Sex == "female") & (df_Titanic.Survived == 1)).select(avg("Age")).show()

+------+------------------+--------+
|Pclass|           AvgFare| MaxFare|
+------+------------------+--------+
|     1| 84.15468749999992|512.3292|
|     3|13.675550101832997|   69.55|
|     2| 20.66218315217391|    73.5|
+------+------------------+--------+

+-----------------+
|         avg(Age)|
+-----------------+
|28.84771573604061|
+-----------------+



In [9]:
from pyspark.sql.functions import min  # Импортируем нужные функции

# количество пассажиров в каждом классе (Pclass).
df_Titanic.groupBy("Pclass").count().show()

# минимальный и максимальный возраст пассажиров
df_Titanic.agg(min("Age").alias("min_age"), max("Age").alias("min_age")).show()

+------+-----+
|Pclass|count|
+------+-----+
|     1|  216|
|     3|  491|
|     2|  184|
+------+-----+

+-------+-------+
|min_age|min_age|
+-------+-------+
|   0.42|   80.0|
+-------+-------+



###  JSON в PySpark

JSON – полуструктурированный формат, поддерживающий вложенные поля. Spark автоматически определяет схему, но для сложных структур её лучше задавать вручную. Вложенные поля обрабатываются через . (например, `user.address.city`).

Функции для работы с JSON:
- from_json() - Преобразует JSON-строку в структуру.
- to_json() - Преобразует структуру в JSON-строку.
- json_tuple() - Извлекает элементы из JSON-строки.
- get_json_object() - Извлекает элемент по JSON-пути.

## 1.3. Базовые операции с DataFrames

Очистка данных. Для того чтобы убрать пропуски (NA/NULL) используют `df.na.fill()` – заполнение пропусков. `df.na.drop()` – удаление строк с пропусками. Дубликаты, `df.dropDuplicates()` – удаление полных дубликатов.

In [10]:
# обработаем пропуски в колонке Age train.csv - df_Titanic
from pyspark.sql.functions import *

# Заполнение пропусков в Age медианным значением
median_age = df_Titanic.approxQuantile("Age", [0.5], 0.01)[0]
df_filled = df_Titanic.na.fill({"Age": median_age})

# Удаление дубликатов (если есть)
df_clean = df_filled.dropDuplicates()

df_clean.show(5)

+-----------+--------+------+--------------------+------+----+-----+-----+--------+-------+-------+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|  Ticket|   Fare|  Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+--------+-------+-------+--------+
|        499|       0|     1|Allison, Mrs. Hud...|female|25.0|    1|    2|  113781| 151.55|C22 C26|       S|
|         11|       1|     3|Sandstrom, Miss. ...|female| 4.0|    1|    1| PP 9549|   16.7|     G6|       S|
|        701|       1|     1|Astor, Mrs. John ...|female|18.0|    1|    0|PC 17757|227.525|C62 C64|       C|
|         55|       0|     1|Ostby, Mr. Engelh...|  male|65.0|    0|    1|  113509|61.9792|    B30|       C|
|        332|       0|     1| Partner, Mr. Austen|  male|45.5|    0|    0|  113043|   28.5|   C124|       S|
+-----------+--------+------+--------------------+------+----+-----+-----+--------+-------+-------+--------+
only showing top 5 

Для сложных условий (например, удаление дубликатов по конкретным колонкам)  
`df.dropDuplicates(["Name", "Pclass"])`

Объединение таблиц (JOIN). Типы JOIN: inner (по умолчанию), left, right, full, cross.

- INNER JOIN (по умолчанию) - возвращает только строки, где есть совпадения в обеих таблицах
- LEFT JOIN (LEFT OUTER JOIN) - возвращает все строки из левой таблицы и совпадающие из правой
- RIGHT JOIN (RIGHT OUTER JOIN) - возвращает все строки из правой таблицы и совпадающие из левой
- FULL JOIN (FULL OUTER JOIN) - возвращает все строки из обеих таблиц
- CROSS JOIN - декартово произведение всех строк обеих таблиц

In [11]:
# Таблица 1: Пассажиры
passengers = spark.createDataFrame([
    (1, "Alice", 28),
    (2, "Bob", 22),
    (3, "Charlie", 30)
], ["passenger_id", "name", "age"])

# Таблица 2: Билеты
tickets = spark.createDataFrame([
    (1, "A123", 150.0),
    (2, "B456", 200.0),
    (4, "C789", 100.0)
], ["passenger_id", "ticket_num", "price"])

# LEFT JOIN (все пассажиры, даже без билетов)
joined_df = passengers.join(tickets, "passenger_id", "left")
joined_df.show()

+------------+-------+---+----------+-----+
|passenger_id|   name|age|ticket_num|price|
+------------+-------+---+----------+-----+
|           1|  Alice| 28|      A123|150.0|
|           2|    Bob| 22|      B456|200.0|
|           3|Charlie| 30|      NULL| NULL|
+------------+-------+---+----------+-----+



**Партиционирование** уменьшает время обработки больших данных.  
`df_repartitioned = df.repartition(4, "Pclass")  # 4 партиции по колонке Pclass`

**Кэширование** ускоряет повторные вычисления  
`df.cache()  # Данные сохраняются в памяти`

## 1.4. SQL-синтаксис в PySpark

Создание временных представлений. **TempView** – виртуальная таблица, доступная только в текущей сессии Spark. **GlobalTempView** – виртуальная таблица, доступная во всех сессиях (используется реже).

In [12]:
# Создание временного представления
df_Titanic.createOrReplaceTempView("titanic")

# Проверка
spark.sql("SHOW TABLES").show()

+---------+---------+-----------+
|namespace|tableName|isTemporary|
+---------+---------+-----------+
|         |  titanic|       true|
+---------+---------+-----------+



In [13]:
# Базовые SQL-запросы

# Запрос через SQL
result = spark.sql("""
    SELECT Name, Age, Pclass 
    FROM titanic 
    WHERE Age > 30 AND Pclass = 1
    ORDER BY Age DESC
""")
result.show(5)

# Средний возраст выживших по классам
spark.sql("""
    SELECT 
        Pclass, 
        AVG(Age) as avg_age,
        COUNT(*) as count
    FROM titanic 
    WHERE Survived = 1
    GROUP BY Pclass
    ORDER BY avg_age
""").show()

+--------------------+----+------+
|                Name| Age|Pclass|
+--------------------+----+------+
|Barkworth, Mr. Al...|80.0|     1|
|Goldschmidt, Mr. ...|71.0|     1|
|Artagaveytia, Mr....|71.0|     1|
|Crosby, Capt. Edw...|70.0|     1|
|Millet, Mr. Franc...|65.0|     1|
+--------------------+----+------+
only showing top 5 rows

+------+------------------+-----+
|Pclass|           avg_age|count|
+------+------------------+-----+
|     3|20.646117647058823|  119|
|     2| 25.90156626506024|   87|
|     1| 35.36819672131148|  136|
+------+------------------+-----+



Комбинация SQL и DataFrame API. Фильтрация через SQL, обработка через DataFrame

In [14]:
# Фильтруем данные через SQL
filtered_df = spark.sql("SELECT * FROM titanic WHERE Embarked = 'S'")

# Агрегация через DataFrame API
from pyspark.sql.functions import avg
filtered_df.groupBy("Pclass").agg(avg("Fare")).show()

+------+------------------+
|Pclass|         avg(Fare)|
+------+------------------+
|     1| 70.36486220472443|
|     3| 14.64408300283288|
|     2|20.327439024390245|
+------+------------------+



In [15]:
# Добавляем столбец через SQL
spark.sql("""
    SELECT *, 
        CASE 
            WHEN Age < 18 THEN 'Child' 
            ELSE 'Adult' 
        END AS age_group
    FROM titanic
""").createOrReplaceTempView("titanic_with_age_group")

# Используем новое представление
spark.sql("SELECT age_group, COUNT(*) FROM titanic_with_age_group GROUP BY age_group").show()

+---------+--------+
|age_group|count(1)|
+---------+--------+
|    Adult|     778|
|    Child|     113|
+---------+--------+



In [16]:
# Оконные функции в SQL.
# Ранжирование пассажиров по стоимости билета
query = """
    SELECT Name, Fare, Pclass, DENSE_RANK() OVER (PARTITION BY Pclass ORDER BY Fare DESC) as fare_rank
    FROM titanic
    WHERE Fare IS NOT NULL
"""
spark.sql(query).show(10)

+--------------------+--------+------+---------+
|                Name|    Fare|Pclass|fare_rank|
+--------------------+--------+------+---------+
|    Ward, Miss. Anna|512.3292|     1|        1|
|Cardeza, Mr. Thom...|512.3292|     1|        1|
|Lesurer, Mr. Gust...|512.3292|     1|        1|
|Fortune, Mr. Char...|   263.0|     1|        2|
|Fortune, Miss. Ma...|   263.0|     1|        2|
|Fortune, Miss. Al...|   263.0|     1|        2|
|   Fortune, Mr. Mark|   263.0|     1|        2|
|Ryerson, Miss. Em...| 262.375|     1|        3|
|"Ryerson, Miss. S...| 262.375|     1|        3|
|Baxter, Mr. Quigg...|247.5208|     1|        4|
+--------------------+--------+------+---------+
only showing top 10 rows

